In [1]:
import os
import posixpath
import gzip
import shutil

from urllib.request import urlretrieve
from urllib.request import urlcleanup

In [34]:
os.getcwd()
os.chdir("../../")
os.getcwd()

'/scratch2/cropbio/stoecker/MuWU/revision_branch/MuWU'

In [112]:
fasta = "ftp://ftp.ensemblgenomes.org/pub/plants/release-50/fasta/zea_mays/dna/Zea_mays.B73_RefGen_v4.dna.toplevel.fa.gz"
#fasta = "/scratch2/cropbio/stoecker/MuWU/data/Col-01_1.fq.gz"
#fasta = "/scratch2/cropbio/stoecker/MuWU/data/gsdsdfsf.fasta"
#annotation = "ftp://ftp.ensemblgenomes.org/pub/plants/release-50/gtf/zea_mays/Zea_mays.B73_RefGen_v4.50.gtf.gz"
annotation = "https://ftp.ncbi.nlm.nih.gov/genomes/refseq/plant/Zea_mays/all_assembly_versions/suppressed/GCF_000005005.2_B73_RefGen_v4/GCF_000005005.2_B73_RefGen_v4_genomic.gbff.gz"
#annotation = "/scratch2/cropbio/stoecker/MuWU/revision_branch/MuWU/gabbawabba.gbff.gz"
assembly_report = "https://ftp.ncbi.nlm.nih.gov/genomes/refseq/plant/Zea_mays/all_assembly_versions/suppressed/GCF_000005005.2_B73_RefGen_v4/GCF_000005005.2_B73_RefGen_v4_assembly_report.txt"

In [110]:
extensions=[".gff", ".gff3", ".gbff", ".gtf", ".dat", ".gff.gz", ".gff3.gz", ".gbff.gz", ".gtf.gz", ".dat.gz"]
genbank=[".gbff", ".dat", ".gbff.gz", ".dat.gz"]
#extensions

In [108]:
def is_url(file):
    return (
        file.startswith("http")
        or file.startswith("ftp")
        or file.startswith("sftp")
    )

In [10]:
def is_valid_annotation_file(annotation):
    for ext in extensions:
        if annotation.endswith("{}".format(ext)):
            return True

In [11]:
def get_file_ext(annotation):
    for ext in extensions:
        if annotation.endswith("{}".format(ext)):
            return ext    

In [12]:
def is_gbff(file):
    for ext in genbank:
        if file.endswith("{}".format(ext)):
            return True

In [21]:
is_gbff(annotation)

In [13]:
def is_gzipped(file):
    return(
        file.endswith(".gz")
    )

In [71]:
def handle_fasta(fasta, annotation):
    if not is_gbff(annotation):
        if is_url(fasta):
            print("Downloading fasta")
            if is_gzipped(fasta):
                urlretrieve(fasta,'resources/genome.fa.gz')
                print("Unpacking fasta")
                with gzip.open('resources/genome.fa.gz', 'rb') as f_in:
                    with open('resources/genome.fa', 'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)
            else:
                urlretrieve(fasta,'resources/genome.fa')
        elif not is_url(fasta):
            print("Gunzipping or linking fasta file already on file system to resources/ dir")
            if is_gzipped(fasta):
                with gzip.open(fasta, 'rb') as f_in:
                    with open('resources/genome.fa', 'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)
            else:
                os.symlink(fasta, "resources/genome.fa")

In [106]:
def gunzip_annotation(annotation_file):
#    with gzip.open('annotation' + get_file_ext(annotation), 'rb') as f_in:
#        with open('annotation' + get_file_ext(annotation).replace(".gz", ""), 'wb') as f_out:
    with gzip.open(annotation_file, 'rb') as f_in:
        with open(annotation_file.replace(".gz", ""), 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [118]:
def handle_annotation(annotation):
    urlcleanup()
    if is_valid_annotation_file(annotation) and not is_gzipped(annotation):
        if is_url(annotation):
            print("Downloading annotation")
            urlretrieve(annotation, 'resources/annotation')
        elif not is_url(annotation):
            printing("Linking annotation")
            os.symlink(annotation, 'resources/annotation' + get_file_ext(annotation))
    if is_valid_annotation_file(annotation) and is_gzipped(annotation):
        if is_url(annotation):
            print("Downloading annotation")
            urlretrieve(annotation, 'resources/annotation.gz')
        elif not is_url(annotation):
            print("Linking annotation")
            os.symlink(annotation, "resources/annotation.gz")
        print("Unpacking annotation")
        gunzip_annotation('resources/annotation.gz')
        print("Done!")

In [25]:
handle_annotation(annotation)

Unpacking annotation


In [119]:
def fasta_annotation_handling(fasta, annotation, assembly_report):
    # check if annotation is valid file - based on file extensions
    if is_valid_annotation_file(annotation):
        print("Annotation file type is supported - continuing:")
        # dealing with the fasta file
        # download/link fasta to resources/ dir
        # we only download/link/use fasta if annotation is NOT in GenBank format
        handle_fasta(fasta, annotation)
        # dealing with the annotation file
        handle_annotation(annotation)
        # if user supplied genbank file also download report
        if is_gbff(annotation):
            print("Supplied annotation in GenBank format - also getting assembly report.")
            if is_url(report):
                urlretrieve(assembly_report, 'resources/assembly_report.txt')
            else:
                os.symlink(assembly_report, "resources/assembly_report.txt")
        print("All done!")
        # create status based upon which we know whether to treat annotation with GenBank or GFF(3)/GTF rules
#        status = ""
#        if is_gbff(annotation):
#            status = "genbank"
#            print("Pipeline will continue with GenBank annotation rules")
#        else:
#            status = "normal"
#            print("Pipeline will continue with GFF(3)/GTF annotation rules")            
#        return status
    else:
        print("Unrecognized annotation file type!")
        print("Valid files are:")
        print(extensions)

In [121]:
fasta_annotation_handling(fasta, annotation, assembly_report)

Annotation file type is supported - continuing:
Unpacking annotation
Done!
Supplied annotation in GenBank format - also getting assembly report.
All done!
